In [2]:
!pip install rasterio
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 62.0 MB/s eta 0:00:00


In [11]:
import rasterio
import rasterio.plot
import geopandas
import numpy as np
from rasterio.enums import Resampling
from google.colab import drive
from osgeo import gdal
import glob
import geopandas as gpd
from shapely import geometry

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
imagery_vrt = "D:/madronus/Land_Cover/redwood/rgb_combined/rgb-imagery.vrt"
nir_vrt = "D:/madronus/Land_Cover/redwood/nir_combined/nir-imagery.vrt"
label_vrt = "D:/madronus/Land_Cover/AOI_2/Classified/classified_raster.tif"
dsm_vrt = "D:/madronus/Land_Cover/redwood/dsm/expanded_demo_dtm.tif"
training_tiles = geopandas.read_file("D:/madronus/Land_Cover/redwood/training_data/validation_processing_grid.gpkg")
target_size = 256

In [12]:
  # Read the shapefile
  gdf = gpd.read_file('/content/drive/MyDrive/pilot_2024_02_01/data/classified_data/lahaina_sub_final.gpkg')

  # Reproject to projected coordinate system
  gdf = gdf.to_crs('EPSG:3857')

  # Get the extent of the shapefile
  total_bounds = gdf.total_bounds

  # Get minX, minY, maxX, maxY
  minX, minY, maxX, maxY = total_bounds

  # Create a fishnet
  x, y = (minX, minY)
  geom_array = []

  # Polygon Size
  square_size = (0.075*256)
  while y <= maxY:
      while x <= maxX:
          geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
          geom_array.append(geom)
          x += square_size
      x = minX
      y += square_size

  fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:3857')

In [14]:
fishnet.to_file('fishnet.gpkg')

In [ ]:
for id in training_tiles.id:
    print(id)
    tile_bounds = training_tiles[training_tiles['id'] == id].envelope.bounds
    imagery_file = rasterio.open(imagery_vrt)
    imagery_window = imagery_file.window(tile_bounds['minx'].values[0], tile_bounds['miny'].values[0],
                                       tile_bounds['maxx'].values[0], tile_bounds['maxy'].values[0])
    imagery_raster = imagery_file.read(window=imagery_window, out_shape=(int(256), int(256)),
                               resampling=Resampling.nearest)
    imagery_file.close()
    output_transform = rasterio.transform.from_bounds(tile_bounds['minx'].values[0], tile_bounds['miny'].values[0],
                                               tile_bounds['maxx'].values[0], tile_bounds['maxy'].values[0],
                                               imagery_raster.shape[1], imagery_raster.shape[2])

    nir_file = rasterio.open(nir_vrt)
    nir_window = nir_file.window(tile_bounds['minx'].values[0], tile_bounds['miny'].values[0],
                                       tile_bounds['maxx'].values[0], tile_bounds['maxy'].values[0])
    nir_raster = nir_file.read(window=nir_window, out_shape=(int(256), int(256)),
                               resampling=Resampling.cubic)
    # nir_image = rasterio.plot.reshape_as_image(nir_raster)
    nir_transform = nir_file.transform
    nir_file.close()

    dsm_file = rasterio.open(dsm_vrt)
    dsm_window = dsm_file.window(tile_bounds['minx'].values[0], tile_bounds['miny'].values[0],
                                       tile_bounds['maxx'].values[0], tile_bounds['maxy'].values[0])
    dsm_raster = dsm_file.read(window=dsm_window, out_shape=(int(256), int(256)),
                               resampling=Resampling.cubic)
    # nir_image = rasterio.plot.reshape_as_image(nir_raster)
    dsm_transform = nir_file.transform
    dsm_file.close()
    print(dsm_raster.min())
    dsm_raster = ((dsm_raster-dsm_raster.min())/15.24)*256
    print(imagery_raster.shape)
    print(nir_raster.shape)
    print(dsm_raster.shape)
    stacked_raster = np.stack([imagery_raster[0], imagery_raster[1], imagery_raster[2], nir_raster[0], dsm_raster[0]])
    # stacked_image = rasterio.plot.reshape_as_image(stacked_raster)



    output_raster = rasterio.open(f'D:/madronus/Land_Cover/redwood/training_data/image-validation/{id}.tif',
                                  'w',
                                  driver='GTiff',
                                  height=256, width= 256,
                                  count=stacked_raster.shape[0], dtype=str(rasterio.uint8),
                                  crs='EPSG:3857',
                                  transform=output_transform)
    for band in range(stacked_raster.shape[0]):
        output_raster.write(stacked_raster[band], band + 1)
    output_raster.close()

    label_file = rasterio.open(label_vrt)
    label_window = label_file.window(tile_bounds['minx'].values[0], tile_bounds['miny'].values[0],
                                       tile_bounds['maxx'].values[0], tile_bounds['maxy'].values[0])
    label_raster = label_file.read(window=label_window)
    # label_image = rasterio.plot.reshape_as_image(label_raster)
    label_transform = label_file.transform
    label_file.close()
    output_raster = rasterio.open(f'D:/madronus/Land_Cover/redwood/training_data/label-validation/{id}.tif',
                                  'w',
                                  driver='GTiff',
                                  height=256, width=256,
                                  count=1, dtype=str(rasterio.uint8),
                                  crs='EPSG:3857',
                                  transform=output_transform)
    for band in range(label_raster.shape[0]):
        output_raster.write(label_raster[band], band + 1)
    output_raster.close()